In [1]:
# Import des librairies
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from preprocessing import preprocess, selection_types_features, data_cleaning_import
import pandas as pd
import numpy as np
import seaborn as sns
from model import train_model


6 Features for Min-Max-Scalering:
 ['Monthly_Grocery_Bill', 'Vehicle_Monthly_Distance_Km', 'Waste_Bag_Weekly_Count', 'How_Long_TV_PC_Daily_Hour', 'How_Many_New_Clothes_Monthly', 'How_Long_Internet_Daily_Hour']
5 Features for One-Hot-Encoding:
 ['Sex', 'Heating_Energy_Source', 'Transport_Vehicle_Type', 'Recycling', 'Cooking_With']
6 Features for Min-Max-Scaling:
 ['Monthly_Grocery_Bill', 'Vehicle_Monthly_Distance_Km', 'Waste_Bag_Weekly_Count', 'How_Long_TV_PC_Daily_Hour', 'How_Many_New_Clothes_Monthly', 'How_Long_Internet_Daily_Hour']
7 Features where we apply Ordinal-Encoding:
 ['Body_Type', 'Diet', 'How_Often_Shower', 'Social_Activity', 'Frequency_of_Traveling_by_Air', 'Waste_Bag_Size', 'Energy_efficiency']


In [2]:
# Charger les données
df = pd.read_csv("raw_data/Carbon_Emission.csv")

In [3]:
data_path= "raw_data/Carbon_Emission.csv"

In [9]:
X = df.drop('CarbonEmission', axis=1)
y = df['CarbonEmission']

In [11]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   -- ------------------------------------- 6.6/124.9 MB 36.6 MB/s eta 0:00:04
   ---- ----------------------------------- 14.7/124.9 MB 38.4 MB/s eta 0:00:03
   ------- -------------------------------- 23.6/124.9 MB 39.3 MB/s eta 0:00:03
   ---------- ----------------------------- 32.2/124.9 MB 39.3 MB/s eta 0:00:03
   ------------- -------------------------- 41.4/124.9 MB 39.9 MB/s eta 0:00:03
   ---------------- ----------------------- 50.6/124.9 MB 40.3 MB/s eta 0:00:02
   ------------------ --------------------- 58.5/124.9 MB 40.1 MB/s eta 0:00:02
   --------------------- ------------------ 67.9/124.9 MB 40.5 MB/s eta 0:00:02
   ------------------------ --------------- 75.5/124.9 MB 39.8 MB/s eta 0:00:02
   --------------------------- ------------ 84.4/124.9 MB 40.2 MB/s eta 0:00:02
   ------------------------------ --------- 93.8/124.9 MB 40.5 MB/s eta 0:00:01
   -------------------------------- ------ 103.5/1

In [10]:
import xgboost
import shap

# train an XGBoost model
X, y = shap.datasets.california()
model = xgboost.XGBRegressor().fit(X, y)

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.Explainer(model)
shap_values = explainer(X)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])

ModuleNotFoundError: No module named 'xgboost'

In [4]:
df, dict_variables_ordinal_categorical=data_cleaning_import(data_path)

In [5]:
def train_model (df, dict_variables_ordinal_categorical):

    # Appel des fonction de preprocessing
    variables_quantitative, variables_ordinal,variables_for_one_hot_encoded = selection_types_features(df)
    cf, X_transformed = preprocess(df, variables_quantitative, variables_ordinal,variables_for_one_hot_encoded, dict_variables_ordinal_categorical)

    #Choix du X et de la target d'entrainement
    X_transformed = X_transformed
    y = df["CarbonEmission"]

    # Diviser les données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, train_size=0.75, random_state=42)

    # Initialiser le modèle avec les meilleurs paramètres trouvés
    best_gbr = GradientBoostingRegressor(
        learning_rate=0.1,
        max_depth=5,
        max_features='sqrt',
        min_samples_leaf=5,
        min_samples_split=2,
        n_estimators=900,
        subsample=0.9,
        random_state=42
    )

    # Entraîner le modèle
    best_gbr.fit(X_train, y_train)

    return best_gbr,cf, X_train, X_test, y_train, y_test


In [6]:
best_gbr,cf, X_train, X_test, y_train, y_test= train_model(df, dict_variables_ordinal_categorical)

6 Features for Min-Max-Scalering:
 ['Monthly_Grocery_Bill', 'Vehicle_Monthly_Distance_Km', 'Waste_Bag_Weekly_Count', 'How_Long_TV_PC_Daily_Hour', 'How_Many_New_Clothes_Monthly', 'How_Long_Internet_Daily_Hour']
5 Features for One-Hot-Encoding:
 ['Sex', 'Heating_Energy_Source', 'Transport_Vehicle_Type', 'Recycling', 'Cooking_With']
6 Features for Min-Max-Scaling:
 ['Monthly_Grocery_Bill', 'Vehicle_Monthly_Distance_Km', 'Waste_Bag_Weekly_Count', 'How_Long_TV_PC_Daily_Hour', 'How_Many_New_Clothes_Monthly', 'How_Long_Internet_Daily_Hour']
7 Features where we apply Ordinal-Encoding:
 ['Body_Type', 'Diet', 'How_Often_Shower', 'Social_Activity', 'Frequency_of_Traveling_by_Air', 'Waste_Bag_Size', 'Energy_efficiency']


In [7]:
# Fonction de visualisation avec SHAP
def visualize_shap(best_gbr, X_test):
    # Utiliser TreeExplainer pour les modèles basés sur des arbres
    explainer = shap.Explainer(best_gbr)

    # Calculer les valeurs SHAP pour les données d'entraînement
    shap_values = explainer.shap_values(X_test)

    # Visualisation globale : importance des features (summary plot)
    shap.summary_plot(shap_values, X_test)

    # Visualisation sous forme de barres pour l'importance moyenne
    shap.summary_plot(shap_values, X_test, plot_type="bar")


In [8]:
# Visualiser SHAP
visualize_shap(best_gbr, X_test)

AttributeError: module 'shap' has no attribute 'Explainer'